# Оценка балла общей облачности (БОО, total cloud cover, TCC)

### Загрузим и предобработаем фотографии

In [7]:
import cv2
import random
import torch
import numpy as np 
import torchvision 
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import datetime
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
import pickle
from tqdm import tqdm
from typing import Tuple, List, Type, Dict, Any
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

In [2]:
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
%cd /content/
!wget https://www.dropbox.com/s/nyy2q2zb1bkw3mi/train.tar.gz

/content
--2021-05-05 15:54:34--  https://www.dropbox.com/s/nyy2q2zb1bkw3mi/train.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nyy2q2zb1bkw3mi/train.tar.gz [following]
--2021-05-05 15:54:35--  https://www.dropbox.com/s/raw/nyy2q2zb1bkw3mi/train.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1e263249014d54a67458c2db85.dl.dropboxusercontent.com/cd/0/inline/BN4o2BKJ4fKYFPz1Z82l951nOHY6jvPx4G7FBi0tbqs4vJzSZorDXxZSQb3ZJBKrpN0uPI49S9E7t4D0YhV7ULyo3NwIkFf-msFX4t9AzNsXpdNcS2JPZZeMpQRF12eNeCGmBMW2NNq9hisL4CmN9N1m/file# [following]
--2021-05-05 15:54:35--  https://uc1e263249014d54a67458c2db85.dl.dropboxusercontent.com/cd/0/inline/BN4o2BKJ4fKYFPz1Z82l951nOHY6jvPx4G7FBi0tbqs4vJzSZorDXxZSQb3ZJBKrpN0uPI49S9E7t4

In [14]:
!wget https://www.dropbox.com/s/a9k38jd1keuwnnv/test.tar.gz

--2021-05-05 15:52:47--  https://www.dropbox.com/s/a9k38jd1keuwnnv/test.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/a9k38jd1keuwnnv/test.tar.gz [following]
--2021-05-05 15:52:48--  https://www.dropbox.com/s/raw/a9k38jd1keuwnnv/test.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc238e5843feb2e535dcd3452df2.dl.dropboxusercontent.com/cd/0/inline/BN5x1_ZIi2hTWtuqwQF8vNjePWToxXXcTRuJmB5b_R14VJUh4IaFpCK9x0mJ_8fkuD5zyiWuZ-3PPWUWNKFmxuTxa-eETKZifiyiBWs1zaIkIVIM9BmvocdojLPLfS6YOM-bEuj-_CeaKS_muab7DaJr/file# [following]
--2021-05-05 15:52:48--  https://uc238e5843feb2e535dcd3452df2.dl.dropboxusercontent.com/cd/0/inline/BN5x1_ZIi2hTWtuqwQF8vNjePWToxXXcTRuJmB5b_R14VJUh4IaFpCK9x0mJ_8fkuD5zyiWuZ-3PPWUWNKFmxuTxa-

In [20]:
!mkdir /content/drive/MyDrive/TCC
!mkdir /content/train
!mkdir /content/test

mkdir: cannot create directory ‘/content/drive/MyDrive/TCC’: File exists
mkdir: cannot create directory ‘/content/train’: File exists
mkdir: cannot create directory ‘/content/test’: File exists


In [16]:
!mv /content/test.tar.gz /content/test
%cd /content/test
!tar xvzf /content/test/test.tar.gz &> /dev/nul

/content/test


In [22]:
!mv /content/train.tar.gz /content/train
%cd /content/train
!tar xvzf /content/train/train.tar.gz &> /dev/nul
%cd /content/drive/MyDrive/TCC

/content/train
/content/drive/MyDrive/TCC


In [17]:
#подавать в конструктор имя корневой папки(train/test)
class CloudDataset(Dataset):
    def __init__(self, root, n_classes, train=True):
        self.root = root
        self.train = train
        self.num_classes = n_classes
        self.data_index = None

        if self.train:
            self.img_transforms = torchvision.transforms.Compose([
                          torchvision.transforms.ToTensor(),
                          torchvision.transforms.RandomRotation(30),
                          torchvision.transforms.RandomVerticalFlip(),
                          torchvision.transforms.RandomHorizontalFlip()
                          #torchvision.transforms.RandomCrop()
                          ])
            self.mask_transforms = torchvision.transforms.Compose([
                          torchvision.transforms.ToTensor()
                          #torchvision.transforms.RandomRotation(10)
                          #torchvision.transforms.RandomCrop()
                          #torchvision.transforms.RandomVerticalFlip()
                          ])
        else:
            self.img_transforms = torchvision.transforms.Compose([
                          torchvision.transforms.ToTensor()]) 
            self.mask_transforms = torchvision.transforms.Compose([
                          torchvision.transforms.ToTensor()])
        
        with open(f"{self.root}/index.pkl", 'rb') as f:
            self.data_index = pickle.load(f)

    def __getitem__(self, idx):
        data_dict = self.data_index[idx]

        #prepare date to correct form to read image correctly
        date = data_dict['observations_dt']
        month = str(date.month)
        day = str(date.day) 

        #change the date format
        if date.month < 10:
            month = "0" + month
        if date.day < 10:
            day = "0" + day

        #prepare pathes to image and mask
        snapshots_dir = f"snapshots-{date.year}-{month}-{day}"
        if data_dict['mission'] == "AI49":
            snapshots_dir = f"ai49-snapshots-{date.year}-{month}-{day}"
        path_to_photo = f"{self.root}/{data_dict['mission']}/snapshots/{snapshots_dir}/{data_dict['jpg_filename']}"
        path_to_mask = f"{self.root}/{data_dict['mask_fname']}"

        #read photo and mask
        cloud_photo = cv2.imread(path_to_photo)
        mask = cv2.imread(path_to_mask)

        #add some useful information about photo to dict
        # information = {}
        # information["mask"] = mask
        # information["date"] = date

        # target = None
        # if 'observed_TCC' in data_dict:
        #     target = data_dict['observed_TCC']
        target = {}
        #target["mask"] = mask
        #target["date"] = date
        target["date"] = np.array([date.year, date.month, date.day])
        name = data_dict['jpg_filename']

        target["label"] = 10
        if 'observed_TCC' in data_dict:
           #target["label"] = one_hot_encode(data_dict['observed_TCC'])
           target["label"] = data_dict['observed_TCC']

        cloud_photo = self.img_transforms(cloud_photo)
        #здесь же аугментировать маску, если нужно 
        #mask = self.mask_transforms(mask)

        for _,value in target.items():
            if value is not None:
              value = torch.tensor(value)
        return cloud_photo, target, name #, information

    def __len__(self):
        return len(self.data_index)

In [18]:
batch_size = 32

In [23]:
train_dataset_full = CloudDataset("/content/train", n_classes=9, train=True)
#(train test split: suffle - True default)

train_size = int(0.8 * len(train_dataset_full))
test_size = len(train_dataset_full) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset_full, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

test_dataset = CloudDataset("/content/test", n_classes=9, train=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [24]:
print("Train len: {}".format(len(train_dataset)))
print("Val len: {}".format(len(val_dataset)))
print("Test len: {}".format(len(test_dataset)))
#plt.imshow(train_dataset_full[0][0].numpy().transpose())

Train len: 73661
Val len: 18416
Test len: 27000


In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [1]:
!nvidia-smi

Wed May  5 15:47:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Multi target network

In [8]:
class Multi_target_Net(nn.Module):
    def __init__(self, network, input_dim, n_classes):
        super().__init__()
        self.basic_net = network
        self.num_classes = n_classes
        self.classifier = nn.Sequential(nn.Linear(input_dim, 128),
                                        nn.GELU(),
                                        nn.Linear(128, 64),
                                        nn.GELU(),
                                        nn.Linear(64, self.num_classes))
        
        self.regressor = nn.Sequential(nn.Linear(input_dim, 128),
                                        nn.GELU(),
                                        nn.Linear(128, 64),
                                        nn.GELU(),
                                        nn.Linear(64, 1))
    def forward(self, input_data):
        basic_output = self.basic_net(input_data)
        classifier_output = self.classifier(basic_output)
        regressor_output = self.regressor(basic_output)

        return classifier_output, regressor_output

In [16]:
def train_model_multi_target(model: torch.nn.Module, 
                train_loader: DataLoader,
                val_loader: DataLoader,
                loss_classifier: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                loss_regressor: torch.nn.Module = torch.nn.MSELoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                initial_lr = 0.01,
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ReduceLROnPlateau,
                lr_scheduler_params: Dict = {},
                max_epochs = 10):

    optimizer = torch.optim.Adam([
                {'params': model.basic_net.parameters(), 'lr': 1e-5},
                {'params': model.classifier.parameters(), 'lr': initial_lr},
                {'params': model.regressor.parameters(), 'lr': initial_lr}
            ], **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)

    best_val_loss = None
    best_epoch = None
    
    loss_history = []
    
    for epoch in range(max_epochs):

        print(f'Epoch {epoch + 1}/{max_epochs}')
        train_epoch_loss_history = train_single_epoch_multi_target(model, optimizer, loss_classifier, loss_regressor, train_loader, epoch)
        loss_history = loss_history+train_epoch_loss_history

        val_metrics = validate_single_epoch_multi_target(model, loss_classifier, loss_regressor, val_loader)

        print(f'Validation metrics: \n{val_metrics}')

        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
            torch.save(model, './best_model.pth')

        torch.save(model.state_dict(), f"./model_on_epoch_{epoch}.pth")

In [17]:
def train_single_epoch_multi_target(model: torch.nn.Module,
                                    optimizer: torch.optim.Optimizer, 
                                    loss_classifier: torch.nn.Module,
                                    loss_regressor: torch.nn.Module,
                                    data_loader: torch.utils.data.DataLoader,
                                    epoch: int):
    
    model.train()
    loss_value = []
    iter = 0
    pbar = tqdm(data_loader)
    for x, y_dict,_ in data_loader:
        x = x.to(device)
        y = y_dict["label"]
        y = y.to(device=device, dtype=torch.int64)
        optimizer.zero_grad()
        output_class, output_regr = model(x)

        loss = loss_classifier(output_class, y) + loss_regressor(output_regr, y.view(-1, 1).float())
        loss.backward()
        loss_value.append(loss.item())
        optimizer.step()
        iter += 1
        if (iter % 500) == 0 :
            torch.save(model, f'./backup_epoch{epoch}_iter{iter}.pth')
        pbar.set_postfix({'loss': loss.item()})
        pbar.update(1)
    pbar.close()      
    
    return loss_value

In [18]:
def validate_single_epoch_multi_target(model: torch.nn.Module,
                                       loss_classifier: torch.nn.Module, 
                                       loss_regressor: torch.nn.Module,
                                       data_loader: torch.utils.data.DataLoader):
    
    val_results = {'loss': 0, 'accuracy': 0}
    
    model.eval()
    pbar = tqdm(data_loader)
    with torch.no_grad():
        for x, y_dict,_ in data_loader:
            x = x.to(device)
            y = y_dict["label"].long()
            y = y.to(device=device, dtype=torch.int64)
            output_class, output_regr = model(x)
            loss = (loss_classifier(output_class, y) + loss_regressor(output_regr, y.view(-1,1).float())).item()
            accuracy = torch.sum(torch.argmax(output_class, dim=1) == y).to(torch.float).item()
            val_results['loss'] += loss
            val_results['accuracy'] += accuracy
            pbar.update(1)
            pbar.set_postfix({'loss': loss, 'accuracy': accuracy})
        val_results['loss'] /= len(data_loader.dataset)
        val_results['accuracy'] /= len(data_loader.dataset)

    pbar.close()
    return val_results

In [3]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=8d23a3b24f2382d59bc1b1d99a091a6a9947c0d47d3d32bc6546d18eb7db92a3
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [4]:
#import torchvision.models as models
#model = models.resnet18(pretrained=True)
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b4')
model

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth



Loaded pretrained weights for efficientnet-b4


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [21]:
#отключение обучения сверточной части
for param in model.parameters():
    param.requires_grad = False

In [22]:
#для обучения сверточной части
for param in model._blocks[31].parameters():
    param.requires_grad = True
for param in model._conv_head.parameters():
    param.requires_grad = True
for param in model._bn1.parameters():
    param.requires_grad = True
for param in model._avg_pooling.parameters():
    param.requires_grad = True
for param in model._dropout.parameters():
    param.requires_grad = True       

In [23]:
model._fc = nn.Linear(in_features=1792, out_features=256)

In [9]:
multi_target_model = Multi_target_Net(network=model, input_dim=256, n_classes=9)

In [12]:
multi_target_model = torch.load("/content/drive/MyDrive/TCC/backup_epoch0_iter2000.pth")

In [25]:
multi_target_model.to(device)

Multi_target_Net(
  (basic_net): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
 

In [ ]:
train_model_multi_target(multi_target_model, 
            train_loader=train_loader, 
            val_loader=val_loader, 
            loss_classifier=torch.nn.CrossEntropyLoss(),
            loss_regressor=torch.nn.MSELoss(),
            lr_scheduler_params={'factor': 0.3, 'patience': 5},
            initial_lr=0.0001,
            max_epochs=1)

  0%|          | 0/2302 [00:00<?, ?it/s]

Epoch 1/1


 90%|████████▉ | 2067/2302 [1:29:34<10:22,  2.65s/it, loss=1.8] 

In [26]:
def test_model_multi_target(model: torch.nn.Module, 
                            data_loader: torch.utils.data.DataLoader):
    output = torch.zeros(1, 9).to(device)
    image_names = []
    model.eval()
    with torch.no_grad():
        for x, _, name in tqdm(data_loader):
            x = x.to(device)
            y, _ = model(x)
            output = torch.cat((output, y), dim=0)
            image_names.append(name)
    return output, image_names

In [27]:
y_pred_test_multi_target, names = test_model_multi_target(multi_target_model, test_loader)

100%|██████████| 844/844 [25:13<00:00,  1.79s/it]


In [28]:
y_pred_test_multi_target_labels = torch.argmax(y_pred_test_multi_target, dim=1)
y_pred_test_multi_target_labels = y_pred_test_multi_target_labels[1:]
result_multi_target = y_pred_test_multi_target_labels.cpu().tolist()

In [29]:
name_list = []
for tup in names:
    for name in tup:
        name_list.append(name)

In [30]:
import pandas as pd
data_mt = dict(jpg_filename=name_list, TCC=result_multi_target)
df = pd.DataFrame(data_mt)
df.to_csv(r'result_model_multi_target.csv', sep=',', index=False)

print(df[0:5])

                               jpg_filename  TCC
0  74c59e4b-6039-4310-bdd8-269da54d16cc.jpg    0
1  c1fd8fc6-d0a3-404b-b533-ea62b9a1b559.jpg    8
2  fa876324-8711-4713-801b-097d626c9cf3.jpg    1
3  404863be-8c7b-4aba-9a11-70e61099068e.jpg    8
4  fabbec27-45fe-435a-b3b7-717a6495bd56.jpg    7


In [ ]:
!ps -aux|grep python

root          79  0.2  0.0      0     0 ?        Z    17:32   0:13 [python3] <defunct>
root         238  0.0  0.0      0     0 ?        Z    17:32   0:00 [python3] <defunct>
root         708  0.0  0.0      0     0 ?        Z    18:45   0:00 [python3] <defunct>
root         821  1.0  0.0      0     0 ?        Zs   18:50   0:09 [python3] <defunct>
root         839  0.1  0.0      0     0 ?        Z    18:50   0:01 [python3] <defunct>
root        1000  0.3  0.0      0     0 ?        Zs   18:59   0:01 [python3] <defunct>
root        1020  0.0  0.0      0     0 ?        Z    18:59   0:00 [python3] <defunct>
root        1075  0.0  0.0      0     0 ?        Z    19:00   0:00 [python3] <defunct>
root        1142  3.3  0.0      0     0 ?        Zs   19:01   0:07 [python3] <defunct>
root        1160  0.0  0.0      0     0 ?        Z    19:01   0:00 [python3] <defunct>
root        1225  0.9  0.4 196208 62676 ?        Sl   19:03   0:01 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0

In [ ]:
!kill -9 79 238 708 821 839 1000 1020 1048 1075 1142 1160